# Preparation

## Imports

In [315]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Functions

In [316]:
def combine_strategy_results(dfs, weights):
    """
    Combine multiple strategy result DataFrames into a single meta-portfolio.

    Parameters
    ----------
    dfs : list of pd.DataFrame
        List of backtest result tables (each must contain ret_net, turnover, tcost, ret_rf, ret_bm, etc.)

    weights : list of float
        Strategy-level weights (must sum to 1.0)

    Returns
    -------
    final_results : pd.DataFrame
        Combined backtest results with aggregated performance and exposure.
    """

    import pandas as pd

    if abs(sum(weights) - 1.0) > 1e-8:
        raise ValueError("Strategy weights must sum to 1.0")

    # --------------------------------------------------------
    # 1) Align on common index (intersection)
    # --------------------------------------------------------
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # --------------------------------------------------------
    # 2) Combine performance metrics
    # --------------------------------------------------------
    ret_net_combined   = sum(w * df["ret_net"]   for w, df in zip(weights, dfs_aligned))
    ret_gross_combined = sum(w * df["ret_gross"] for w, df in zip(weights, dfs_aligned))
    ret_bm_combined    = sum(w * df["ret_bm"]    for w, df in zip(weights, dfs_aligned))
    turnover_combined  = sum(w * df["turnover"]  for w, df in zip(weights, dfs_aligned))
    tcost_combined     = sum(w * df["tcost"]     for w, df in zip(weights, dfs_aligned))
    ret_rf_combined    = sum(w * df["ret_rf"]    for w, df in zip(weights, dfs_aligned))

    # Performance-only DataFrame
    final_results = pd.DataFrame({
        "ret_net": ret_net_combined,
        "ret_gross": ret_gross_combined,
        "ret_bm": ret_bm_combined,
        "turnover": turnover_combined,
        "tcost": tcost_combined,
        "ret_rf": ret_rf_combined,
    })

    # --------------------------------------------------------
    # 3) Collect all weight columns across all strategies
    # --------------------------------------------------------
    region_cols = sorted({
        col for df in dfs_aligned for col in df.columns if col.startswith("w_")
    })

    # --------------------------------------------------------
    # 4) Ensure all strategies have all weight columns, fill missing with 0
    # --------------------------------------------------------
    for i, df in enumerate(dfs_aligned):
        for reg in region_cols:
            if reg not in df.columns:
                dfs_aligned[i][reg] = 0.0

    # --------------------------------------------------------
    # 5) Combine region weights using strategy weights
    # --------------------------------------------------------
    for reg in region_cols:
        final_results[reg] = sum(w * df[reg] for w, df in zip(weights, dfs_aligned))

    return final_results


In [317]:
def apply_fx_hedging(backtest_df, hedge_ratio, hedge_returns):
    """
    Adds FX hedging P&L on top of an existing backtest.

    - USD is treated as the base currency and will NOT be hedged.
    - The same hedge_ratio is used for long and short positions.
    """

    df = backtest_df.copy()

    # Extract region names from weight columns
    regions = [col.replace("w_", "") for col in df.columns if col.startswith("w_")]

    # Remove USD from hedging universe
    regions = [r for r in regions if r != "US"]

    # Align hedge data with Date index
    hedge_ratio = hedge_ratio.reindex(df.index)
    hedge_returns = hedge_returns.reindex(df.index)

    hedge_pnl = pd.Series(0.0, index=df.index)

    for region in regions:
        w_col = f"w_{region}"

        # Check hedge ratio availability (should match non-US regions)
        if region not in hedge_ratio.columns:
            print(f"⚠ Region {region} not in hedge_ratio — skipping.")
            continue

        # Check hedge return availability
        if region not in hedge_returns.columns:
            print(f"⚠ Region {region} not in hedge_returns — skipping.")
            continue

        # Use the same hedge ratio for long and short positions
        hedge_mult = hedge_ratio[region]

        # Hedge contribution
        contrib = df[w_col] * hedge_mult * hedge_returns[region]

        hedge_pnl += contrib

    # Add hedged P&L to ret_net
    df["ret_net"] = df["ret_net"] + hedge_pnl

    return df


In [318]:
def combine_macro_weights(dfs, weights):
    """
    Combine multiple macro-strategy weight tables into a single weighted average.

    Parameters
    ----------
    dfs : list of DataFrame
        Weight tables (w_AU, w_CH, ...)
    weights : list of float
        Strategy weights, must sum to 1.0

    Returns
    -------
    DataFrame
        Combined weighted sum of all weights
    """
    import pandas as pd
    import numpy as np

    # Align all to the same index
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # Identify region columns
    region_cols = [c for c in dfs_aligned[0].columns if c.startswith("w_")]

    # Weighted sum
    combined = pd.DataFrame(index=common_index)

    for col in region_cols:
        combined[col] = sum(w * df[col] for w, df in zip(weights, dfs_aligned))

    return combined


In [319]:
def combine_two_weight_tables(df1, df2, weight1=0.5, weight2=0.5):
    """
    Combine two weight tables (e.g., trend + macro) using weighted average.
    Missing regions in either table are treated as 0.
    """
    import pandas as pd

    # 1. Align indices
    idx = df1.index.union(df2.index)
    df1a = df1.reindex(idx)
    df2a = df2.reindex(idx)

    # 2. Align columns (regions)
    cols = sorted(set(df1a.columns).union(df2a.columns))
    df1a = df1a.reindex(columns=cols).fillna(0.0)
    df2a = df2a.reindex(columns=cols).fillna(0.0)

    # 3. Weighted combination
    combined = weight1 * df1a + weight2 * df2a

    return combined


## Data

### Strategies

In [320]:
# --- Fundamental Data ---
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)

# --- Macro Data ---
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)
macroRS_bond = pd.read_csv("../Results/macroRS_bond.csv", index_col=0, parse_dates=True)
macroRS_rates = pd.read_csv("../Results/macroRS_rates.csv", index_col=0, parse_dates=True)
macroRS_fx = pd.read_csv("../Results/macroRS_fx.csv", index_col=0, parse_dates=True)

macroIT_equity = pd.read_csv("../Results/macroIT_equity.csv", index_col=0, parse_dates=True)
macroIT_bond = pd.read_csv("../Results/macroIT_bond.csv", index_col=0, parse_dates=True)
macroIT_rates = pd.read_csv("../Results/macroIT_rates.csv", index_col=0, parse_dates=True)
macroIT_fx = pd.read_csv("../Results/macroIT_fx.csv", index_col=0, parse_dates=True)

macroMP_equity = pd.read_csv("../Results/macroMP_equity.csv", index_col=0, parse_dates=True)
macroMP_bond = pd.read_csv("../Results/macroMP_bond.csv", index_col=0, parse_dates=True)
macroMP_rates = pd.read_csv("../Results/macroMP_rates.csv", index_col=0, parse_dates=True)
macroMP_fx = pd.read_csv("../Results/macroMP_fx.csv", index_col=0, parse_dates=True)    

macroBC_equity = pd.read_csv("../Results/macroBC_equity.csv", index_col=0, parse_dates=True)
macroBC_bond = pd.read_csv("../Results/macroBC_bond.csv", index_col=0, parse_dates=True)
macroBC_rates = pd.read_csv("../Results/macroBC_rates.csv", index_col=0, parse_dates=True)
macroBC_fx = pd.read_csv("../Results/macroBC_fx.csv", index_col=0, parse_dates=True)


# --- Trend Following Data ---
trend_equity = pd.read_csv("../Results/trend_equity.csv", index_col=0, parse_dates=True)
trend_bond = pd.read_csv("../Results/trend_bond.csv", index_col=0, parse_dates=True)
trend_rates = pd.read_csv("../Results/trend_rates.csv", index_col=0, parse_dates=True)
trend_fx = pd.read_csv("../Results/trend_fx.csv", index_col=0, parse_dates=True)

In [321]:
def extract_clean_weights(df):
    # 1) Nur die weight-Spalten nehmen
    weight_cols = [c for c in df.columns if c.startswith("w_")]
    
    # 2) Gewichtstabelle extrahieren
    w = df[weight_cols].copy()
    
    # 3) Prefix "w_" entfernen
    w.columns = [c.replace("w_", "") for c in weight_cols]
    
    return w

# Clean weights for both strategies
fundamental_equity = extract_clean_weights(fundamental_equity)

trend_equity = extract_clean_weights(trend_equity)
trend_bond = extract_clean_weights(trend_bond)
trend_rates = extract_clean_weights(trend_rates)
trend_fx = extract_clean_weights(trend_fx)

macroRS_equity = extract_clean_weights(macroRS_equity)
macroRS_bond = extract_clean_weights(macroRS_bond)
macroRS_rates = extract_clean_weights(macroRS_rates)
macroRS_fx = extract_clean_weights(macroRS_fx)
macroIT_equity = extract_clean_weights(macroIT_equity)
macroIT_bond = extract_clean_weights(macroIT_bond)
macroIT_rates = extract_clean_weights(macroIT_rates)
macroIT_fx = extract_clean_weights(macroIT_fx)
macroMP_equity = extract_clean_weights(macroMP_equity)
macroMP_bond = extract_clean_weights(macroMP_bond)
macroMP_rates = extract_clean_weights(macroMP_rates)
macroMP_fx = extract_clean_weights(macroMP_fx)
macroBC_equity = extract_clean_weights(macroBC_equity)
macroBC_bond = extract_clean_weights(macroBC_bond)
macroBC_rates = extract_clean_weights(macroBC_rates)
macroBC_fx = extract_clean_weights(macroBC_fx)



### Asset Data

In [322]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    "../Data/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_returns = equity_prices.pct_change()
equity_returns

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


In [323]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    "../Data/Bond Data.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


In [324]:
# --- Load Rates Futures ---
rates_futures = pd.read_excel(
    "../Data/Interest Rates Data.xlsx",
    index_col = 0,
    parse_dates = True
)
rates_futures.index = pd.to_datetime(rates_futures.index)
rates_futures.index = rates_futures.index + pd.offsets.MonthEnd(0)

rates_returns = rates_futures.pct_change()
rates_returns

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,-0.002432,NaN,0.000844,0.000524,0.013047,NaN,0.001312
1998-04-30,-0.000509,NaN,-0.000730,-0.000209,0.013644,NaN,0.000302
1998-05-31,0.000000,NaN,0.000422,-0.000733,-0.014970,NaN,0.000554
...,...,...,...,...,...,...,...
2025-07-31,0.000000,-0.001019,-0.002281,0.000000,0.000000,-0.001840,0.000000
2025-08-31,0.000000,-0.000459,-0.000207,0.000000,-0.009700,-0.002765,0.000000
2025-09-30,0.000000,0.000051,-0.001759,0.000000,-0.004021,-0.001232,0.000000


In [325]:
# --- Load Currency Prices ---
fx_data = pd.read_excel(
    "../Data/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.index = fx_data.index + pd.offsets.MonthEnd(0)
fx_returns = fx_data
fx_returns



,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,0.000000
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,0.000000
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,0.000000
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,0.000000
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,0.000000
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


### Riskfree Data

In [326]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    "../Data/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_30000\3539450603.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Benchmark Data

In [327]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    "../Data/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['Full Benchmark']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.006737
2025-08-31    0.023840
2025-09-30    0.027309
2025-10-31    0.015763
2025-11-30    0.003025
Freq: ME, Name: Full Benchmark, Length: 468, dtype: float64

### Factor Data

In [328]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    "../Data/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_30000\383609709.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,UMD,BAB,RMW,CMA
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,0.0745,-0.0056,-0.0184,0.0189
1980-02-29,-0.0123,-0.0162,0.0059,0.0789,0.0571,-0.0095,0.0292
1980-03-31,-0.1289,-0.0697,-0.0096,-0.0958,0.0605,0.0182,-0.0105
1980-04-30,0.0396,0.0105,0.0103,-0.0048,0.0145,-0.0218,0.0034
1980-05-31,0.0526,0.0200,0.0038,-0.0118,-0.0311,0.0043,-0.0063
...,...,...,...,...,...,...,...
2025-06-30,0.0486,-0.0002,-0.0160,-0.0264,-0.0246,-0.0320,0.0145
2025-07-31,0.0198,-0.0015,-0.0127,-0.0096,-0.0842,-0.0029,-0.0208
2025-08-31,0.0185,0.0488,0.0442,-0.0354,0.0391,-0.0068,0.0207


### Hedging Data

In [329]:
# Read the CSV
fx_predictions = pd.read_csv("../Data/FX_predictions.csv", index_col=0, parse_dates=["Date"])
fx_predictions.index = fx_predictions.index + pd.offsets.MonthEnd(0)

# Keep only Date, Region, Hedge Ratio
df_fx = fx_predictions[["Region", "Hedge_Ratio_Next_Month"]]

# Pivot to wide format
hedge_ratio = df_fx.pivot_table(
    index=df_fx.index,
    columns="Region",
    values="Hedge_Ratio_Next_Month"
)

if "GB" in hedge_ratio.columns:
    hedge_ratio = hedge_ratio.rename(columns={"GB": "UK"})

# Ensure proper column order (optional)
df_fhedge_ratiox_wide = hedge_ratio.sort_index(axis=1)

hedge_ratio.dropna(inplace=True)
hedge_ratio


Region,AU,CH,EM,EU,JP,UK
Date,,,,,,
2000-03-31,1.00,0.00,0.75,0.00,0.50,0.75
2000-04-30,1.00,0.00,0.75,0.00,0.50,0.75
2000-05-31,0.75,0.00,0.75,0.00,0.75,0.75
2000-06-30,1.00,0.00,0.75,0.00,0.75,1.00
2000-07-31,1.00,0.00,0.75,0.50,0.75,1.00
...,...,...,...,...,...,...
2025-04-30,0.50,0.00,0.00,0.00,0.00,0.00
2025-05-31,0.50,0.50,0.00,0.00,0.00,0.00
2025-06-30,0.50,0.75,0.00,0.00,0.00,0.00


In [330]:
# --- Load Currency Prices ---
hedge_return = pd.read_excel(
    "../Data/FX Data.xlsx",
    sheet_name = 'HEDGE RETURN',
    index_col = 0,
    parse_dates = True
)
hedge_return.index = pd.to_datetime(hedge_return.index)
hedge_return.index = hedge_return.index + pd.offsets.MonthEnd(0)
hedge_returns = hedge_return
hedge_returns

,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,0.062505,0.000000,0.044448,0.028124,-0.043550,0.000000
1989-02-28,-0.020885,0.000000,-0.025663,0.002575,0.095372,0.000000
1989-03-31,0.067754,0.000000,0.048472,0.030821,-0.027844,0.000000
1989-04-30,0.009532,0.000000,0.006180,-0.004816,0.025641,0.000000
1989-05-31,0.022300,0.000000,0.070709,0.068691,0.047106,0.000000
...,...,...,...,...,...,...
2025-06-30,-0.033185,-0.036774,0.003389,-0.020141,-0.022884,-0.004021
2025-07-31,0.027679,0.033666,0.048025,0.038372,0.024318,0.009627
2025-08-31,-0.010759,-0.021636,-0.021495,-0.022178,-0.017316,-0.009796


## Combine Macro Strategies

In [331]:
macro_equity = combine_macro_weights(
    dfs=[
        macroRS_equity,
        macroIT_equity,
        macroMP_equity,
        macroBC_equity
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)
macro_bond = combine_macro_weights(
    dfs=[
        macroRS_bond,
        macroIT_bond,
        macroMP_bond,
        macroBC_bond
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_rates = combine_macro_weights(
    dfs=[
        macroRS_rates,
        macroIT_rates,
        macroMP_rates,
        macroBC_rates
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_fx = combine_macro_weights(
    dfs=[
        macroRS_fx,
        macroIT_fx,
        macroMP_fx,
        macroBC_fx
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

In [ ]:
macro_equity, macro_bond, macro_rates, macro_fx

""
Date
2001-10-31
2001-11-30
2001-12-31
2002-01-31
2002-02-28
...
2025-06-30
2025-07-31
2025-08-31


# Combined Strategies

## 1. Trend + Macro

### Individual Backtests

In [332]:
bond_macro_trend = combine_two_weight_tables(trend_bond, macro_bond, 0.5, 0.5)

bond_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = bond_macro_trend,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [333]:
equity_macro_trend = combine_two_weight_tables(trend_equity, macro_equity, 0.5, 0.5)

equity_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = equity_macro_trend,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [334]:
fx_macro_trend = combine_two_weight_tables(trend_fx, macro_fx, 0.5, 0.5)

fx_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = fx_macro_trend,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [335]:
rates_macro_trend = combine_two_weight_tables(trend_rates, macro_rates, 0.5, 0.5)

rates_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = rates_macro_trend,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [336]:
macro_trend = combine_strategy_results(
    dfs=[
        equity_macro_trend_results,
        bond_macro_trend_results,
        fx_macro_trend_results,
        rates_macro_trend_results
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.003241,0.003353,0.000000,0.561895,0.000112,0.0044,0.127055,-0.148672,0.024754,-0.085779,0.118921,0.070508,-0.103434
2000-01-31,0.006564,0.006689,0.000000,0.626086,0.000125,0.0041,-0.137767,-0.153910,0.125517,0.015140,0.143217,0.018548,-0.004056
2000-02-29,0.001544,0.001613,-0.001213,0.342744,0.000069,0.0043,-0.055906,-0.004985,0.120172,0.016502,0.169270,-0.077648,-0.165793
2000-03-31,-0.002957,-0.002892,0.054429,0.326677,0.000065,0.0047,-0.008258,-0.056135,0.078301,-0.120139,0.275351,-0.019433,-0.152578
2000-04-30,0.002625,0.002686,-0.038301,0.305630,0.000061,0.0046,-0.074209,-0.047976,0.124180,-0.018198,0.130285,-0.088748,-0.022647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.000700,0.000865,0.038273,0.822134,0.000164,0.0034,-0.215163,0.149778,0.273025,-0.309786,-0.178645,0.165983,0.115672
2025-07-31,-0.008644,-0.008341,0.006737,1.516320,0.000303,0.0034,-0.105366,-0.165736,-0.181005,0.352585,-0.035583,0.202182,-0.075417
2025-08-31,-0.000223,-0.000024,0.023840,0.994338,0.000199,0.0038,-0.000949,0.090173,0.289134,-0.049948,-0.229261,0.168384,-0.267558


### Hedged Performance

In [337]:
macro_trend_hedged = apply_fx_hedging(
    backtest_df=macro_trend,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)

hedge_returns.to_excel("../Results/JUST_TRYING_SOMETHING.xlsx")


In [338]:
macro_trend_hedged.dropna(inplace=True)
macro_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-03-31,-0.012299,-0.002892,0.054429,0.326677,0.000065,0.0047,-0.008258,-0.056135,0.078301,-0.120139,0.275351,-0.019433,-0.152578
2000-04-30,0.001686,0.002686,-0.038301,0.305630,0.000061,0.0046,-0.074209,-0.047976,0.124180,-0.018198,0.130285,-0.088748,-0.022647
2000-05-31,-0.005434,-0.004570,-0.011620,0.360130,0.000072,0.0050,0.106745,-0.198821,0.115609,-0.070812,0.164942,-0.099719,-0.022516
2000-06-30,0.003899,-0.000167,0.030136,0.345706,0.000069,0.0040,-0.132590,-0.196606,0.095988,-0.048569,0.211104,-0.007959,0.078465
2000-07-31,0.000023,-0.000931,-0.023792,0.344011,0.000069,0.0048,-0.074024,-0.133966,0.124640,-0.067669,0.180922,-0.091921,0.061089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.016696,-0.016240,0.014553,0.953523,0.000191,0.0035,0.021650,-0.268438,0.311113,-0.315990,0.090959,0.075148,0.069319
2025-05-31,0.003362,0.002711,0.045232,0.586780,0.000117,0.0038,-0.358583,-0.086589,0.282144,-0.266271,0.020066,0.347733,0.064211
2025-06-30,-0.000566,0.000865,0.038273,0.822134,0.000164,0.0034,-0.215163,0.149778,0.273025,-0.309786,-0.178645,0.165983,0.115672


In [339]:
macro_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.2090,1.9566
Arithm Avg Xs Return,4.4000,0.1476
Std Xs Returns,9.8457,2.7293
Sharpe Arithmetic,0.4469,0.0541
Geom Avg Total Return,5.8780,1.9346
Geom Avg Xs Return,4.0555,0.1121
Sharpe Geometric,0.4119,0.0411
Min Xs Return,-10.2114,-3.5408
Max Xs Return,8.6715,3.3422
Skewness,-0.4262,0.1593


In [340]:
pmp.run_factor_regression(macro_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.9566
Arithm Avg Xs Return,0.1476
Std Xs Returns,2.7293
Sharpe Arithmetic,0.0541
Geom Avg Total Return,1.9346
Geom Avg Xs Return,0.1121
Sharpe Geometric,0.0411
Min Xs Return,-3.5408
Max Xs Return,3.3422
Skewness,0.1593


### Unhedged Performance

In [341]:
macro_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.2707,2.0822
Arithm Avg Xs Return,4.4143,0.2258
Std Xs Returns,9.7797,2.7337
Sharpe Arithmetic,0.4514,0.0826
Geom Avg Total Return,5.9498,2.0622
Geom Avg Xs Return,4.0792,0.1917
Sharpe Geometric,0.4171,0.0701
Min Xs Return,-10.2114,-4.1619
Max Xs Return,8.6715,3.4146
Skewness,-0.4281,-0.2134


In [342]:
pmp.run_factor_regression(macro_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.0822
Arithm Avg Xs Return,0.2258
Std Xs Returns,2.7337
Sharpe Arithmetic,0.0826
Geom Avg Total Return,2.0622
Geom Avg Xs Return,0.1917
Sharpe Geometric,0.0701
Min Xs Return,-4.1619
Max Xs Return,3.4146
Skewness,-0.2134


## 2.  Trend + Fundamental

### Individual Backtests

In [343]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [344]:
trend_equity_results = pmp.run_cc_strategy_drift(
    weights      = trend_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [345]:
trend_bond_results = pmp.run_cc_strategy_drift(
    weights      = trend_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [346]:
trend_fx_results = pmp.run_cc_strategy_drift(
    weights      = trend_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [347]:
trend_rates_results = pmp.run_cc_strategy_drift(
    weights      = trend_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [348]:
fundamental_trend = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        trend_equity_results,
        trend_bond_results,
        trend_fx_results,
        trend_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.004243,0.004370,-0.028188,0.631961,0.000126,0.0051,0.016098,-0.004669,0.112666,-0.196956,0.101797,-0.395974,0.209847
2000-12-31,0.008916,0.009066,0.024349,0.749905,0.000150,0.0050,-0.052431,-0.015828,0.091324,-0.073737,-0.231936,-0.065868,0.159068
2001-01-31,0.000559,0.000685,0.010556,0.632294,0.000126,0.0054,0.121810,-0.192222,0.154510,-0.070253,0.013217,0.207804,-0.090056
2001-02-28,-0.010529,-0.010452,-0.048796,0.383529,0.000077,0.0038,0.117043,-0.055170,0.138128,-0.002742,-0.037489,0.079427,-0.016070
2001-03-31,-0.005573,-0.005491,-0.049222,0.406578,0.000081,0.0042,0.092864,0.007369,-0.009450,0.043556,-0.189520,0.196653,-0.098073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.005400,-0.005190,0.038273,1.046415,0.000209,0.0034,-0.167962,0.018744,-0.060811,-0.432992,-0.146837,0.263671,0.577690
2025-07-31,0.007840,0.008189,0.006737,1.742286,0.000348,0.0034,0.023285,-0.277292,-0.460941,0.181153,-0.199346,0.324630,0.418508
2025-08-31,0.008381,0.008683,0.023840,1.507061,0.000301,0.0038,0.072181,0.135623,0.096019,-0.160217,-0.177424,0.297382,-0.217284


### Hedged Performance

In [349]:
fundamental_trend_hedged = apply_fx_hedging(
    backtest_df=fundamental_trend,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [350]:
fundamental_trend_hedged.dropna(inplace=True)
fundamental_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.003432,0.004370,-0.028188,0.631961,0.000126,0.0051,0.016098,-0.004669,0.112666,-0.196956,0.101797,-0.395974,0.209847
2000-12-31,0.017928,0.009066,0.024349,0.749905,0.000150,0.0050,-0.052431,-0.015828,0.091324,-0.073737,-0.231936,-0.065868,0.159068
2001-01-31,0.001579,0.000685,0.010556,0.632294,0.000126,0.0054,0.121810,-0.192222,0.154510,-0.070253,0.013217,0.207804,-0.090056
2001-02-28,-0.011130,-0.010452,-0.048796,0.383529,0.000077,0.0038,0.117043,-0.055170,0.138128,-0.002742,-0.037489,0.079427,-0.016070
2001-03-31,-0.001687,-0.005491,-0.049222,0.406578,0.000081,0.0042,0.092864,0.007369,-0.009450,0.043556,-0.189520,0.196653,-0.098073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.007513,-0.005639,0.014553,1.506957,0.000301,0.0035,0.128063,-0.289127,-0.032297,-0.405362,0.276553,0.285768,0.056646
2025-05-31,0.017858,0.017660,0.045232,1.170464,0.000234,0.0038,-0.184178,-0.295400,0.018752,-0.368754,-0.038916,0.380459,0.479096
2025-06-30,-0.003944,-0.005190,0.038273,1.046415,0.000209,0.0034,-0.167962,0.018744,-0.060811,-0.432992,-0.146837,0.263671,0.577690


In [351]:
fundamental_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.4592,3.1266
Arithm Avg Xs Return,4.7579,1.4253
Std Xs Returns,9.7996,5.5588
Sharpe Arithmetic,0.4855,0.2564
Geom Avg Total Return,6.1454,3.0130
Geom Avg Xs Return,4.4322,1.2998
Sharpe Geometric,0.4523,0.2338
Min Xs Return,-10.2114,-4.6034
Max Xs Return,8.6715,5.4759
Skewness,-0.4574,0.1045


In [352]:
pmp.run_factor_regression(fundamental_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,3.1266
Arithm Avg Xs Return,1.4253
Std Xs Returns,5.5588
Sharpe Arithmetic,0.2564
Geom Avg Total Return,3.0130
Geom Avg Xs Return,1.2998
Sharpe Geometric,0.2338
Min Xs Return,-4.6034
Max Xs Return,5.4759
Skewness,0.1045


### Unhedged Performance

In [353]:
fundamental_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.5885,2.9024
Arithm Avg Xs Return,4.8705,1.1844
Std Xs Returns,9.7763,7.3247
Sharpe Arithmetic,0.4982,0.1617
Geom Avg Total Return,6.2840,2.6464
Geom Avg Xs Return,4.5539,0.9163
Sharpe Geometric,0.4658,0.1251
Min Xs Return,-10.2114,-24.2449
Max Xs Return,8.6715,6.2514
Skewness,-0.4663,-4.8793


In [354]:
pmp.run_factor_regression(fundamental_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.9024
Arithm Avg Xs Return,1.1844
Std Xs Returns,7.3247
Sharpe Arithmetic,0.1617
Geom Avg Total Return,2.6464
Geom Avg Xs Return,0.9163
Sharpe Geometric,0.1251
Min Xs Return,-24.2449
Max Xs Return,6.2514
Skewness,-4.8793


## 3. Fundamental + Macro

### Individual Backtests

In [355]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [356]:
macro_equity_results = pmp.run_cc_strategy_drift(
    weights      = macro_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [357]:
macro_bond_results = pmp.run_cc_strategy_drift(
    weights      = macro_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [358]:
macro_fx_results = pmp.run_cc_strategy_drift(
    weights      = macro_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [359]:
macro_rates_results = pmp.run_cc_strategy_drift(
    weights      = macro_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [360]:
fundamental_macro = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_equity_results,
        macro_bond_results,
        macro_fx_results,
        macro_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_macro

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2002-03-31,0.003593,0.003635,0.022050,0.208895,0.000042,0.0013,0.000000,0.054905,0.000000,0.075364,0.293798,-0.351011,-0.175822
2002-04-30,0.016605,0.016657,-0.001331,0.256592,0.000051,0.0015,0.000000,0.193862,0.000000,0.007466,0.217819,-0.269533,-0.030262
2002-05-31,0.027636,0.027683,0.014872,0.232319,0.000046,0.0014,0.000000,0.053074,0.000000,0.007090,0.354032,-0.104924,-0.075821
2002-06-30,-0.007755,-0.007701,-0.007289,0.270819,0.000054,0.0013,0.000000,-0.055004,0.000000,-0.028866,0.148182,-0.052697,0.065387
2002-07-31,-0.018506,-0.018419,-0.035635,0.434745,0.000087,0.0015,0.000000,-0.040038,0.000000,0.018298,-0.199947,0.365980,0.057062
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.014832,0.014949,0.045232,0.583684,0.000117,0.0038,0.174406,-0.208811,-0.263392,-0.102483,-0.058982,0.032725,0.414885
2025-06-30,-0.006100,-0.006055,0.038273,0.224280,0.000045,0.0034,0.047202,-0.131034,-0.333837,-0.123206,0.031808,0.097688,0.462018
2025-07-31,0.016484,0.016530,0.006737,0.225966,0.000045,0.0034,0.128651,-0.111556,-0.279936,-0.171433,-0.163764,0.122448,0.493925


### Hedged Performance

In [361]:
fundamental_macro_hedged = apply_fx_hedging(
    backtest_df=fundamental_macro,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [362]:
fundamental_macro_hedged.dropna(inplace=True)
fundamental_macro_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2002-03-31,0.002374,0.003635,0.022050,0.208895,0.000042,0.0013,0.000000,0.054905,0.000000,0.075364,0.293798,-0.351011,-0.175822
2002-04-30,0.007545,0.016657,-0.001331,0.256592,0.000051,0.0015,0.000000,0.193862,0.000000,0.007466,0.217819,-0.269533,-0.030262
2002-05-31,0.017276,0.027683,0.014872,0.232319,0.000046,0.0014,0.000000,0.053074,0.000000,0.007090,0.354032,-0.104924,-0.075821
2002-06-30,-0.004034,-0.007701,-0.007289,0.270819,0.000054,0.0013,0.000000,-0.055004,0.000000,-0.028866,0.148182,-0.052697,0.065387
2002-07-31,-0.022768,-0.018419,-0.035635,0.434745,0.000087,0.0015,0.000000,-0.040038,0.000000,0.018298,-0.199947,0.365980,0.057062
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,0.009183,0.010601,0.014553,0.553434,0.000111,0.0035,0.106413,-0.020689,-0.343411,-0.089372,0.185595,0.210620,-0.012672
2025-05-31,0.014497,0.014949,0.045232,0.583684,0.000117,0.0038,0.174406,-0.208811,-0.263392,-0.102483,-0.058982,0.032725,0.414885
2025-06-30,-0.003379,-0.006055,0.038273,0.224280,0.000045,0.0034,0.047202,-0.131034,-0.333837,-0.123206,0.031808,0.097688,0.462018


In [363]:
fundamental_macro_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.3752,2.7739
Arithm Avg Xs Return,5.7918,1.1905
Std Xs Returns,9.7609,4.6671
Sharpe Arithmetic,0.5934,0.2551
Geom Avg Total Return,7.1191,2.6982
Geom Avg Xs Return,5.5255,1.1047
Sharpe Geometric,0.5661,0.2367
Min Xs Return,-10.2114,-4.2473
Max Xs Return,8.6715,4.0198
Skewness,-0.4911,-0.0460


In [364]:
pmp.run_factor_regression(fundamental_macro_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.7739
Arithm Avg Xs Return,1.1905
Std Xs Returns,4.6671
Sharpe Arithmetic,0.2551
Geom Avg Total Return,2.6982
Geom Avg Xs Return,1.1047
Sharpe Geometric,0.2367
Min Xs Return,-4.2473
Max Xs Return,4.0198
Skewness,-0.0460


### Unhedged Performance

In [365]:
fundamental_macro.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.4649,3.4476
Arithm Avg Xs Return,5.8731,1.8558
Std Xs Returns,9.7516,4.6430
Sharpe Arithmetic,0.6023,0.3997
Geom Avg Total Return,7.2155,3.3920
Geom Avg Xs Return,5.6134,1.7899
Sharpe Geometric,0.5756,0.3855
Min Xs Return,-10.2114,-3.2348
Max Xs Return,8.6715,4.2548
Skewness,-0.4969,0.2226


In [366]:
pmp.run_factor_regression(fundamental_macro, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,3.4476
Arithm Avg Xs Return,1.8558
Std Xs Returns,4.6430
Sharpe Arithmetic,0.3997
Geom Avg Total Return,3.3920
Geom Avg Xs Return,1.7899
Sharpe Geometric,0.3855
Min Xs Return,-3.2348
Max Xs Return,4.2548
Skewness,0.2226


## 4. Fundamental + Macro + Trend

### Individual Backtests

#### Macro + Trend (Weight = 2/3)

In [367]:
macro_trend # see above

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.003241,0.003353,0.000000,0.561895,0.000112,0.0044,0.127055,-0.148672,0.024754,-0.085779,0.118921,0.070508,-0.103434
2000-01-31,0.006564,0.006689,0.000000,0.626086,0.000125,0.0041,-0.137767,-0.153910,0.125517,0.015140,0.143217,0.018548,-0.004056
2000-02-29,0.001544,0.001613,-0.001213,0.342744,0.000069,0.0043,-0.055906,-0.004985,0.120172,0.016502,0.169270,-0.077648,-0.165793
2000-03-31,-0.002957,-0.002892,0.054429,0.326677,0.000065,0.0047,-0.008258,-0.056135,0.078301,-0.120139,0.275351,-0.019433,-0.152578
2000-04-30,0.002625,0.002686,-0.038301,0.305630,0.000061,0.0046,-0.074209,-0.047976,0.124180,-0.018198,0.130285,-0.088748,-0.022647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.000700,0.000865,0.038273,0.822134,0.000164,0.0034,-0.215163,0.149778,0.273025,-0.309786,-0.178645,0.165983,0.115672
2025-07-31,-0.008644,-0.008341,0.006737,1.516320,0.000303,0.0034,-0.105366,-0.165736,-0.181005,0.352585,-0.035583,0.202182,-0.075417
2025-08-31,-0.000223,-0.000024,0.023840,0.994338,0.000199,0.0038,-0.000949,0.090173,0.289134,-0.049948,-0.229261,0.168384,-0.267558


#### Fundamental (Weight = 1/3)

In [368]:
fundamental_equity_results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.027872,0.027872,-0.028188,0.000000,0.000000,0.0051,0.000000,0.000000,0.000000,0.000000,-0.624071,0.151240,0.177581
2000-12-31,0.018812,0.018919,0.024349,0.534371,0.000107,0.0050,0.000000,0.000000,0.000000,0.000000,-0.147962,-0.255471,0.025404
2001-01-31,-0.004175,-0.004093,0.010556,0.408307,0.000082,0.0054,0.000000,0.000000,0.000000,0.000000,0.311027,0.024310,-0.051180
2001-02-28,-0.030311,-0.030293,-0.048796,0.091500,0.000018,0.0038,0.000000,0.000000,0.000000,0.000000,0.309513,0.144530,-0.017178
2001-03-31,-0.006545,-0.006501,-0.049222,0.219393,0.000044,0.0042,0.000000,0.000000,0.000000,0.000000,0.335054,0.002969,-0.246744
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.012200,-0.012110,0.038273,0.448561,0.000090,0.0034,0.094403,-0.262069,-0.667673,-0.246411,0.195375,0.063617,0.924035
2025-07-31,0.032969,0.033059,0.006737,0.451932,0.000090,0.0034,0.257303,-0.223111,-0.559872,-0.342865,0.244895,-0.327527,0.987850
2025-08-31,0.017209,0.017414,0.023840,1.025445,0.000205,0.0038,0.146260,0.090900,-0.386231,-0.220539,0.257997,0.103672,0.100548


### Combined

In [369]:
combined = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_trend
    ],
    weights=[0.25, 0.75]
)

combined

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,-0.000301,-0.000207,-0.028188,0.473971,0.000095,0.0051,0.012074,-0.003502,0.084500,-0.147717,0.057443,-0.218972,0.135188
2000-12-31,0.004336,0.004435,0.024349,0.495633,0.000099,0.0050,-0.039323,-0.011871,0.068493,-0.055303,-0.142018,-0.030906,0.116125
2001-01-31,0.000941,0.001026,0.010556,0.423182,0.000085,0.0054,0.091358,-0.144166,0.115883,-0.052690,0.006874,0.116975,-0.061145
2001-02-28,-0.004108,-0.004052,-0.048796,0.276210,0.000055,0.0038,0.087782,-0.041378,0.103596,-0.002056,-0.046183,0.020881,-0.009905
2001-03-31,-0.003361,-0.003306,-0.049222,0.277510,0.000056,0.0042,0.069648,0.005526,-0.007088,0.032667,-0.142511,0.105608,-0.042712
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.002525,-0.002379,0.038273,0.728741,0.000146,0.0034,-0.137772,0.046817,0.037851,-0.293942,-0.118080,0.173331,0.317763
2025-07-31,0.001759,0.002009,0.006737,1.250223,0.000250,0.0034,-0.014699,-0.180080,-0.275722,0.178723,-0.108569,0.212860,0.190399
2025-08-31,0.004135,0.004335,0.023840,1.002115,0.000200,0.0038,0.035853,0.090354,0.120293,-0.092595,-0.146027,0.190787,-0.175531


### Hedged Performance

In [370]:
combined_hedged = apply_fx_hedging(
    backtest_df=combined,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [371]:
combined_hedged.dropna(inplace=True)
combined_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.000035,-0.000207,-0.028188,0.473971,0.000095,0.0051,0.012074,-0.003502,0.084500,-0.147717,0.057443,-0.218972,0.135188
2000-12-31,0.010444,0.004435,0.024349,0.495633,0.000099,0.0050,-0.039323,-0.011871,0.068493,-0.055303,-0.142018,-0.030906,0.116125
2001-01-31,0.001133,0.001026,0.010556,0.423182,0.000085,0.0054,0.091358,-0.144166,0.115883,-0.052690,0.006874,0.116975,-0.061145
2001-02-28,-0.004903,-0.004052,-0.048796,0.276210,0.000055,0.0038,0.087782,-0.041378,0.103596,-0.002056,-0.046183,0.020881,-0.009905
2001-03-31,-0.000867,-0.003306,-0.049222,0.277510,0.000056,0.0042,0.069648,0.005526,-0.007088,0.032667,-0.142511,0.105608,-0.042712
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.007931,-0.006879,0.014553,0.991859,0.000198,0.0035,0.069444,-0.211673,0.061630,-0.281679,0.161016,0.161671,0.045653
2025-05-31,0.009770,0.009508,0.045232,0.731927,0.000146,0.0038,-0.181735,-0.169347,0.079912,-0.250945,-0.014441,0.277163,0.255601
2025-06-30,-0.002114,-0.002379,0.038273,0.728741,0.000146,0.0034,-0.137772,0.046817,0.037851,-0.293942,-0.118080,0.173331,0.317763


In [372]:
combined_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.4592,2.4631
Arithm Avg Xs Return,4.7579,0.7617
Std Xs Returns,9.7996,3.2184
Sharpe Arithmetic,0.4855,0.2367
Geom Avg Total Return,6.1454,2.4371
Geom Avg Xs Return,4.4322,0.7239
Sharpe Geometric,0.4523,0.2249
Min Xs Return,-10.2114,-2.3907
Max Xs Return,8.6715,3.3717
Skewness,-0.4574,0.2120


In [373]:
pmp.run_factor_regression(combined_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.4631
Arithm Avg Xs Return,0.7617
Std Xs Returns,3.2184
Sharpe Arithmetic,0.2367
Geom Avg Total Return,2.4371
Geom Avg Xs Return,0.7239
Sharpe Geometric,0.2249
Min Xs Return,-2.3907
Max Xs Return,3.3717
Skewness,0.2120


### Unhedged Performance

In [374]:
combined.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.5885,2.3565
Arithm Avg Xs Return,4.8705,0.6385
Std Xs Returns,9.7763,4.1104
Sharpe Arithmetic,0.4982,0.1553
Geom Avg Total Return,6.2840,2.2930
Geom Avg Xs Return,4.5539,0.5629
Sharpe Geometric,0.4658,0.1369
Min Xs Return,-10.2114,-13.1629
Max Xs Return,8.6715,3.6558
Skewness,-0.4663,-4.3567


In [375]:
pmp.run_factor_regression(combined, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.3565
Arithm Avg Xs Return,0.6385
Std Xs Returns,4.1104
Sharpe Arithmetic,0.1553
Geom Avg Total Return,2.2930
Geom Avg Xs Return,0.5629
Sharpe Geometric,0.1369
Min Xs Return,-13.1629
Max Xs Return,3.6558
Skewness,-4.3567
